# Semantic SfM: single scene on synthetic data generated from Kubric 

### 1a. Generate synthetic data
We developed a tool to generate synthetic data using Kubric. Please follow the instructions on the Github repo: https://github.com/ZhiangChen/data_generator_3d

### 1b. Organize data
Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

```
semantic_SfM/data
    ├── kubric_0
        ├── photos
        │       ├── 0.png
        │       ├── 1.png
        │       ├── ...
        │       └── 323.png
        ├── reconstructions
        │       ├── camera_poses.npy
        │       └── combined_point_cloud.las   
        ├── segmentations_gt
        │       ├── 0.npy
        │       ├── 0.png
        │       └── ...
        └── associations
                └── depth
                        ├── 0.npy
                        ├── 0.png
                        └── ...
```

In [2]:
import os

scene_dir = '../../data/kubric_0'
pointcloud_path = os.path.join(scene_dir, 'reconstructions', 'combined_point_cloud.las')
associations_folder_path = os.path.join(scene_dir, 'associations')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
photos_folder_path = os.path.join(scene_dir, 'photos')

### 2. Segment images using SAM 

#### 2a. SAM segmentation


In [23]:
from ssfm.image_segmentation import ImageSegmentation

In [ ]:
sam_params = {}
sam_params['model_name'] = 'sam2'
sam_params['model_path'] = '../../semantic_SfM/sam2/sam2.1_hiera_large.pt'
sam_params['device'] = 'cuda:1'
sam_params['points_per_side'] = 32
sam_params['points_per_batch'] = 128
sam_params['pred_iou_thresh'] = 0.6
sam_params['stability_score_offset'] = 0.5
sam_params['box_nms_thresh'] = 0.6
sam_params['use_m2m'] = True


image_paths = [os.path.join(scene_dir, 'photos', image) for image in os.listdir(os.path.join(scene_dir, 'photos'))]

# sort images based on the values of keyimages in file names
image_paths = sorted(image_paths, key=lambda x: int(x.split('/')[-1].split('.')[0]))


In [12]:
run_segmentation = False

if run_segmentation:
    image_segmentor = ImageSegmentation(sam_params)   
    image_segmentor.batch_predict(image_paths, segmentations_folder_path, save_overlap=True)

Processing image 1/285.
Processing image 2/285.
Processing image 3/285.
Processing image 4/285.
Processing image 5/285.
Processing image 6/285.
Processing image 7/285.
Processing image 8/285.
Processing image 9/285.
Processing image 10/285.
Processing image 11/285.
Processing image 12/285.
Processing image 13/285.
Processing image 14/285.
Processing image 15/285.
Processing image 16/285.
Processing image 17/285.
Processing image 18/285.
Processing image 19/285.
Processing image 20/285.
Processing image 21/285.
Processing image 22/285.
Processing image 23/285.
Processing image 24/285.
Processing image 25/285.
Processing image 26/285.
Processing image 27/285.
Processing image 28/285.
Processing image 29/285.
Processing image 30/285.
Processing image 31/285.
Processing image 32/285.
Processing image 33/285.
Processing image 34/285.
Processing image 35/285.
Processing image 36/285.
Processing image 37/285.
Processing image 38/285.
Processing image 39/285.
Processing image 40/285.
Processin

#### 2b. Mask filtering

In [3]:
from ssfm.simple_mask_filter import AreaFilter
area_filter = AreaFilter()

In [4]:
segmentations_folder_path = os.path.join(scene_dir, 'segmentations_filtered')

config = {'area_lower_threshold': 100,
'segmentation_folder_path': '../../data/kubric_0/segmentations',
'output_folder_path': segmentations_folder_path,
'num_processes':8}

area_filter(config)

100%|██████████| 285/285 [00:05<00:00, 52.50it/s]


### 3. Create projection associations

In [5]:
from ssfm.probabilistic_projection import *

In [6]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=1.8)
pointcloud_projector.read_kubric_camera_parameters(scene_dir)
pointcloud_projector.read_pointcloud(pointcloud_path)

In [7]:
# get image list
photo_folder_path = os.path.join(scene_dir, 'photos')
image_list = [f for f in os.listdir(photo_folder_path) if f.endswith('.png')]
# sort image list based on the number in the file name
image_list.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))

In [8]:
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, associations_folder_path, num_workers=16, save_depth=False)

Processing frames: 100%|██████████| 285/285 [00:13<00:00, 20.56it/s]


In [9]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [10]:
print(associations_folder_path)
print(segmentations_folder_path)
smc_solver = KeyimageAssociationsBuilder(image_list, associations_folder_path, segmentations_folder_path)
smc_solver.build_associations()

../../data/kubric_0/associations
../../data/kubric_0/segmentations_filtered


100%|██████████| 285/285 [00:01<00:00, 211.60it/s]


In [11]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 687711     | 32.23                |
| Number of points covered by less than or equal to 1 image    | 802259     | 37.59                |
| Number of points covered by less than or equal to 3 images   | 1098365    | 51.47                |
| Number of points covered by less than or equal to 5 images   | 1406791    | 65.92                |


### 5. Estimate memory usage

In [11]:
from ssfm.memory_calculator import memory_calculator

In [12]:
# pointcloud file
las_file = pointcloud_path
# image file sample
image_file = os.path.join(photos_folder_path, image_list[0])
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       |    0.00048828125     |
| Pixel2point association for each image |     0.0009765625     |
| Point2pixel association for each image | 0.007866773754358292 |
|                                        |                      |
|      Segmentation for all images       |    0.13916015625     |
| Pixel2point association for all images |     0.2783203125     |
| Point2pixel association for all images |  2.242030519992113   |
|          pc_segmentation_ids           | 0.03933386877179146  |
|         pc_segmentation_probs          | 0.03933386877179146  |
|          keyimage_association          |  0.5605076299980283  |
|                 Total                  |  3.2986863562837243  |
+----------------------------------------+----------------------+


### 6. Run object registration

In [12]:
from ssfm.object_registration import *
from ssfm.post_processing import *

keyimage_associations_file_name = 'associations_keyimage.npy'

In [13]:
print(pointcloud_path)
print(associations_folder_path)
print(segmentations_folder_path)

../../data/kubric_0/reconstructions/combined_point_cloud.las
../../data/kubric_0/associations
../../data/kubric_0/segmentations_filtered


In [14]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentations_folder_path, associations_folder_path, keyimage_associations_file_name=keyimage_associations_file_name, image_list=image_list)

# Run object registration
obr.object_registration(iou_threshold=0.2, save_semantics=True)

Processing images: 100%|██████████| 285/285 [04:52<00:00,  1.03s/it]


In [15]:
image_id = 284
semantics_folder_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path, remove_small_N=500, nearest_interpolation=500)

Before removing small semantics: 
maximum of semantics:  5080
number of unique semantics:  534
After removing small semantics: 
number of unique semantics:  207


In [16]:
print(save_las_path)

# read las 
pc = laspy.read(save_las_path)
# get the semantics from the intensity
semantics = pc.intensity
semantics_ids = np.unique(semantics)
print('number of unique semantics: ', len(semantics_ids))
# print the number of points for each semantics
K = 0
for i in semantics_ids:
    n = np.sum(semantics == i)
    if n < 100:
        print('semantics id: ', i, ' number of points: ', n)
        K += 1

print('number of semantics with less than 100 points: ', K)

../../data/kubric_0/associations/semantics/semantics_284.las
number of unique semantics:  206
number of semantics with less than 100 points:  0


In [17]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_shuffled.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  206


In [18]:
# 500, 500
from validation import Validator
validator = Validator(
        save_las_path,
        pointcloud_path,
    )

results = validator.validate(np.arange(0.5, 1.0, 0.05))
mAP = results['AP']
mAR = results['AR']
print('AP: ', mAP, ' AR: ', mAR)
mAP = np.sum(mAP) / len(mAP)
mAR = np.sum(mAR) / len(mAR)
print('mAP: ', mAP, ' mAR: ', mAR)

Unique semantics in prediction:  206
Unique semantics in ground truth:  201
AP:  [0.9271844660194175, 0.912621359223301, 0.9077669902912622, 0.9029126213592233, 0.8737864077669902, 0.8543689320388349, 0.8106796116504854, 0.7815533980582524, 0.6553398058252428, 0.35436893203883496]  AR:  [0.9502487562189055, 0.9353233830845771, 0.9303482587064676, 0.9253731343283582, 0.8955223880597015, 0.8756218905472637, 0.8308457711442786, 0.8009950248756219, 0.6716417910447762, 0.36318407960199006]
mAP:  0.7980582524271844  mAR:  0.817910447761194
